# PART 2 – PYTHON - Teleperformance Application
Developed by: Diego Eusse Naranjo <br>
Personal ID: C.C 1037655233 <br>
Date: Jan $26^{th}$ - 22

 ## Point I.   
 Below is the source code for a function called ‘get_sql_string’.

In [20]:
def get_sql_string(stores):
    store_names = [x.split(', ')[0] for x in stores]
    store_names = [x.replace(' ', '_') for x in store_names]
    store_regions = [x.split(',')[1] for x in stores]
    locations = store_names + store_regions
    columns = ['sales_' + x.lower() for x in locations]
    return ', '.join(columns)

**1.**	There’s an error in line 4. What is it and how will you correct it?

In [21]:
s1 = ['Teleperformance LTDA, Colombia', 'Teleperformance SA, Peru']
get_sql_string(s1)

'sales_teleperformance_ltda, sales_teleperformance_sa, sales_ colombia, sales_ peru'

In line 4, split method is separating the x string just taking the comma into account and the blank space is being left within the strings, so store_regions is storing the name of the locations after a blank space. For Example: ['  Colombia', '  Peru'] . In the end, this causes the algorithm to return an incorrect string. For example: ‘sales_ colombia, sales_ peru’, which could make some trouble within SQL.

This can be solved by adding the blank space within the split string parameter, as follows:

In [22]:
def get_sql_string(stores):
    store_names = [x.split(', ')[0] for x in stores]
    store_names = [x.replace(' ', '_') for x in store_names]
    store_regions = [x.split(', ')[1] for x in stores]
    locations = store_names + store_regions
    columns = ['sales_' + x.lower() for x in locations]
    return ', '.join(columns)

In [23]:
s1 = ['Teleperformance LTDA, Colombia', 'Teleperformance SA, Peru']
get_sql_string(s1)

'sales_teleperformance_ltda, sales_teleperformance_sa, sales_colombia, sales_peru'

As can be seen above, the strings do not have the blank space anymore.

---

**2.**	Assuming this bug was fixed, what would be returned if the following command was executed:

In [24]:
my_stores = ['Fulham Palace Rd, Hammersmith', 'Crown St, Reading', 'Leavesden Green, Watford']
get_sql_string(my_stores)

'sales_fulham_palace_rd, sales_crown_st, sales_leavesden_green, sales_hammersmith, sales_reading, sales_watford'

---

**3.**	Write a python function that: <br>
•	Accepts a list of strings as input, <br>
•	Drops the strings including numbers or special characters, <br>
•	Prints the top 10 string(s) with maximum length sorted A-Z.<br>
Test it by including a list of items that would meet the requirements asked above. Paste a snippet of your code and output.


In [77]:
def string_process(strings):
    filtered_strings = [x for x in strings if x.isalpha()] #Takes only the alphabetic strings
    filtered_strings.sort(key=len, reverse=True)           #Sorts the list according to the strings length, starting from the longest
    filtered_strings = filtered_strings[0:10]              #Takes the top 10 longest string(s)
    filtered_strings.sort(key=str.lower)                                #Sorts the strings alphabetically A-Z
    return filtered_strings

In [78]:
text = "Hello, My NaMe 1$ D1360 &  I, wAnt 2 W0Rk 4 Teleperformance . I'm aN ArtiFicial InteLligence SpecialliSt fr0m Medellin , Antioquia . I hav3 experience w1thin Data Processing & Automation ."
string_list = text.split(' ')
string_process(string_list)

['Antioquia',
 'ArtiFicial',
 'Automation',
 'experience',
 'InteLligence',
 'Medellin',
 'NaMe',
 'Processing',
 'SpecialliSt',
 'Teleperformance']

---

## Point II.   
We oversee performing a scoring process daily, over the thousands of customers we contact every day per campaign. We assign a weight to different characteristics (based on predefined parameters), calculate the score of this customer (only on the day it enters our database), and save it in a column named initial_score. <br>

Create a python function that uses this value to perform daily reductions based on the following, and saves the reduced score in column reduced_score: <br>

* Customers between 0 and 15 days (column ‘Days’) will reduce only 10 points per day.
* Customers between 16 and 30 days (column ‘Days’) will reduce 50 points per day.
* Score in customers with more than 30 days (column ‘Days’) will be immediately reduced to zero.
* Maximum initial_score will always be 1,200 points.
* Once the reduced_score reaches 0, it will print a message saying: “Customer ID#____ has reached score 0 in ____ days”. Use data in columns ‘Days’ and ‘CustomerID’ to complete it.

Let's create a simulation dataset that meets the given conditions:

In [38]:
import datetime
import pandas as pd
import random

start_date = datetime.datetime(2021, 12, 25)
end_date = datetime.datetime.now()

days_between_dates = (end_date - start_date).days
print(days_between_dates)
data = []

for i in range(100):
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + datetime.timedelta(days=random_number_of_days)
    initial_score = random.randint(600, 1200)
    reduced_score = random.randint(0, initial_score)
    data.append([i, random_date,(end_date - random_date).days, initial_score, reduced_score])

df = pd.DataFrame(data, columns = ["CustomerID", "InitialDate", "Days", "InitialScore", "ReducedScore"])
df

31


,CustomerID,InitialDate,Days,InitialScore,ReducedScore
0,0,2021-12-29,27,1161,40
1,1,2021-12-25,31,889,716
2,2,2022-01-07,18,1097,253
3,3,2022-01-15,10,959,875
4,4,2022-01-04,21,766,146
...,...,...,...,...,...
95,95,2022-01-04,21,689,599
96,96,2021-12-28,28,627,63
97,97,2021-12-30,26,839,251
98,98,2022-01-12,13,807,340


In [39]:
def reduce_by_days(df):
    for i in range(0, len(df)):
        df.loc[i, ("Days")] = (datetime.datetime.now() - df.loc[i, ("InitialDate")]).days
        days = df.loc[i, ("Days")]
        value = 10 if days < 15 else (50 if days < 30 else df.loc[i, ("InitialScore")])
        df.loc[i, ("ReducedScore")] = df.loc[i, ("ReducedScore")] + value if days <= 30 else value
        if df.loc[i, ("InitialScore")] <= df.loc[i, ("ReducedScore")]:
            print("Customer ID " + str(df.loc[i, ("CustomerID")]) + " has reached score 0 in " + str(df.loc[i, ("Days")]) + " days")
    return df
df2 = reduce_by_days(df)
df2

Customer ID 1 has reached score 0 in 31 days
Customer ID 5 has reached score 0 in 30 days
Customer ID 11 has reached score 0 in 30 days
Customer ID 26 has reached score 0 in 30 days
Customer ID 38 has reached score 0 in 31 days
Customer ID 39 has reached score 0 in 31 days
Customer ID 50 has reached score 0 in 22 days
Customer ID 75 has reached score 0 in 31 days
Customer ID 80 has reached score 0 in 22 days
Customer ID 81 has reached score 0 in 30 days
Customer ID 86 has reached score 0 in 31 days


,CustomerID,InitialDate,Days,InitialScore,ReducedScore
0,0,2021-12-29,27,1161,90
1,1,2021-12-25,31,889,889
2,2,2022-01-07,18,1097,303
3,3,2022-01-15,10,959,885
4,4,2022-01-04,21,766,196
...,...,...,...,...,...
95,95,2022-01-04,21,689,649
96,96,2021-12-28,28,627,113
97,97,2021-12-30,26,839,301
98,98,2022-01-12,13,807,350


In [31]:
df2.iloc[6]

CustomerID                        6
InitialDate     2021-12-26 00:00:00
Days                             30
InitialScore                    628
ReducedScore                    855
Name: 6, dtype: object

---

### Point III. 
Classes give us the ability to create more complicated data structures that contain arbitrary content. Follow these steps to create a Class for Triangles:
1.	Create a class, Triangle. Arguments in its __init__() method: self, angle1, angle2, and angle3.
2.	Create a variable named number_of_sides and set it equal to 3.
3.	Create a method named check_angles. It should return True if the sum of self.angle1, self.angle2, and self.angle3 is equal 180, and False otherwise.
4.	Create a variable named my_triangle and set it equal to a new instance of your Triangle class. Pass it three angles that sum to 180 (e.g. 90, 30, 60).
5.	Print out my_triangle.number_of_sides and print out my_triangle.check_angles().

In [74]:
class Triangle:
    
    def __init__(self, angle1, angle2, angle3):
        self.number_of_sides = 3
        self.angle1 = angle1
        self.angle2 = angle2
        self.angle3 = angle3
    
    def check_angles(self):
        if(self.angle1 + self.angle2 + self.angle3 == 180):
            return True
        else:
            return False

In [75]:
my_triangle = Triangle(40, 60, 80)
print(my_triangle.number_of_sides)
print(my_triangle.check_angles())

3
True


---

### Point IV. 
Suppose the following four csv files are available to perform data analysis on the transaction history of one of our retail customers:

**transactions.csv** (2 years of data) <br>
customer_id <br>
store_id <br>
date_id <br>
ticket_id <br>
product_id <br>
unit <br>
sales <br>

**stores.csv** <br>
store_id <br>
store_name <br>
state <br>
banner <br>

**customer.csv** <br>
customer_id <br>
age <br>
gender <br>
household_size <br>

**products.csv** <br>
product_id <br>
product_name <br>
brand <br>
supplier <br>
department (food, non-food etc) <br>
category (water, chocolate, clothes etc) <br>



As dataframes are not provided, some random testing .csv files were created. I start by reading them:

In [2]:
transactions_df = pd.read_csv("transactions.csv", sep = ";")
stores_df = pd.read_csv("stores.csv", sep = ";")
customer_df = pd.read_csv("customer.csv", sep = ";")
products_df = pd.read_csv("products.csv", sep = ";")

**1.** Write a code to sort customer table by customer_id (ascending order) and remove duplicates:

Let's see the original disorganized dataframe and the duplicated row at the end of the customers dataset:

In [3]:
customer_df.tail()

,customer_id,age,gender,household_size
6,4,53,F,5
7,2,60,F,5
8,1,32,M,2
9,10,50,F,5
10,10,50,F,5


And let's prove the code for this problem:

In [6]:
customer_df = pd.read_csv("customer.csv", sep = ";")
sorted_customers = customer_df.sort_values(by=['customer_id'], ascending=True)
dropped_customers = sorted_customers.drop_duplicates(subset=['customer_id'])
dropped_customers

,customer_id,age,gender,household_size
8,1,32,M,2
7,2,60,F,5
2,3,61,M,4
6,4,53,F,5
4,5,54,M,3
5,6,41,F,4
0,7,63,M,2
1,8,61,F,2
3,9,34,M,1
9,10,50,F,5


---

**2.**	Create a table named transaction_cube merging all tables.

In [7]:
import pandas as pd

transactions_df = pd.read_csv("transactions.csv", sep = ";")
stores_df = pd.read_csv("stores.csv", sep = ";")
customer_df = pd.read_csv("customer.csv", sep = ";")
products_df = pd.read_csv("products.csv", sep = ";")

df1 = transactions_df.merge(stores_df, on='store_id')
df2 = df1.merge(customer_df, on='customer_id')
transaction_cube = df2.merge(products_df, on='product_id')
transaction_cube

,customer_id,store_id,date_id,ticket_id,product_id,unit,sales,store_name,state,banner,age,gender,household_size,product_name,brand,supplier,department,category
0,1,3,10,1,7,4,50,C,S3,B3,32,M,2,P7,BR7,Z7,DEP3,CAT2
1,1,5,1,9,7,3,69,E,S3,B5,32,M,2,P7,BR7,Z7,DEP3,CAT2
2,10,4,1,5,7,1,147,D,S2,B4,50,F,5,P7,BR7,Z7,DEP3,CAT2
3,10,4,1,5,7,1,147,D,S2,B4,50,F,5,P7,BR7,Z7,DEP3,CAT2
4,6,2,2,4,7,3,20,B,S2,B2,41,F,4,P7,BR7,Z7,DEP3,CAT2
5,1,1,10,14,1,4,186,A,S1,B1,32,M,2,P1,BR1,Z1,DEP1,CAT3
6,7,1,1,3,1,3,100,A,S1,B1,63,M,2,P1,BR1,Z1,DEP1,CAT3
7,10,2,4,12,1,4,163,B,S2,B2,50,F,5,P1,BR1,Z1,DEP1,CAT3
8,10,2,4,12,1,4,163,B,S2,B2,50,F,5,P1,BR1,Z1,DEP1,CAT3
9,1,5,1,7,4,3,160,E,S3,B5,32,M,2,P4,BR4,Z4,DEP4,CAT3


**3.**	Create a table containing customer_id (s) existing in the transactions table but not in the customer table

In [8]:
import pandas as pd

transactions_df = pd.read_csv("transactions.csv", sep = ";")
customer_df = pd.read_csv("customer.csv", sep = ";")
transaction_customers = transactions_df [['customer_id']]
customer_ids = customer_df[['customer_id']]

filtered_customers = transaction_customers.loc[~transaction_customers['customer_id'].isin(customer_ids ['customer_id'])].copy()
filtered_customers

,customer_id
15,11


**4.**	Create a table named customer_summary with the following variables:
* Customer_id
* Banner
* Category
* Department
* Total_sales
* Total_units
* Average_ticket
* Last_visit ---



In [11]:
import pandas as pd

customer_summary = transaction_cube.copy()
customer_summary = customer_summary[['customer_id', 'banner', 'category', 'department', 'sales', 'unit', 'ticket_id', 'date_id']]
customer_summary = customer_summary.groupby(['customer_id', 'banner', 'category', 'department']).agg({'sales':'sum', 'unit':'sum', 'ticket_id':'mean', 'date_id':'max'})
customer_summary.columns = ['Total_sales', 'Total_units', 'Average_ticket', 'Last_visit']
customer_summary

Total_sales  Total_units  \
customer_id banner category department                             
1           B1     CAT3     DEP1                186            4   
            B3     CAT2     DEP3                 50            4   
            B5     CAT2     DEP2                 57            2   
                            DEP3                 69            3   
                   CAT3     DEP4                160            3   
2           B2     CAT1     DEP4                155            1   
6           B2     CAT2     DEP3                 20            3   
7           B1     CAT3     DEP1                100            3   
9           B1     CAT1     DEP4                146            4   
            B2     CAT2     DEP2                170            1   
            B3     CAT4     DEP3                105            3   
10          B1     CAT1     DEP2                478            2   
            B2     CAT1     DEP4                366            4   
                   CAT3     DEP1                326            8   
            B4     CAT2     DEP3                294            2   

                                        Average_ticket  Last_visit  
customer_id banner category department                              
1           B1     CAT3     DEP1                    14          10  
            B3     CAT2     DEP3                     1          10  
            B5     CAT2     DEP2                    15          10  
                            DEP3                     9           1  
                   CAT3     DEP4                     7           1  
2           B2     CAT1     DEP4                     8           1  
6           B2     CAT2     DEP3                     4           2  
7           B1     CAT3     DEP1                     3           1  
9           B1     CAT1     DEP4                    13           7  
            B2     CAT2     DEP2                     6           8  
            B3     CAT4     DEP3                     2           3  
10          B1     CAT1     DEP2                    11           6  
            B2     CAT1     DEP4                    10          10  
                   CAT3     DEP1                    12           4  
            B4     CAT2     DEP3                     5           1

**5.**	Create a table named customer_metrics with the following variables:
* Customer_id
* Month_year
* Banner
* Category
* Department
* Total_sales
* Total_units
* Visits_count (# tickets)
* Visits_count_customer (# tickets regardless of product, category, or department)
* Products_count
* Products_count_customer (# products regardless of banner)
* Median_Price
* Distinct_Stores


In [26]:
import datetime
customer_metrics = transaction_cube.copy()
customer_metrics = customer_metrics[['customer_id', 'banner', 'category', 'department', 'sales', 'unit', 'ticket_id', 'date_id', 'product_id', 'store_id']]
customer_metrics['date_id'] = customer_metrics['date_id'].map(lambda x: datetime.datetime.utcfromtimestamp(x))
array = customer_metrics['date_id'].map(lambda x: str(x.month) + '_' + str(x.year))
customer_metrics['month_year'] = array
customer_metrics = customer_metrics.groupby(['customer_id', 'banner', 'category', 'department', 'store_id', 'month_year']).agg({'sales':'sum', 'unit':'sum', 'ticket_id':'count', 'product_id':'count', 'sales':'mean'})
df1 = customer_metrics.groupby(level=[0, 1, 4, 5]).agg({'ticket_id':'count'})
df1.columns = ['Visits_count_customer']
customer_metrics = customer_metrics.join(df1)
df2 = customer_metrics.groupby(level=[0, 2, 3, 4, 5]).agg({'product_id':'count'})
df2.columns = ['Products_count_customer']
customer_metrics = customer_metrics.join(df2)
customer_metrics.columns = ['Total_sales', 'Total_units', 'Visits_count', 'Products_count', 'Visits_count_customer', 'Products_count_customer']
customer_metrics

Total_sales  \
customer_id store_id month_year category department banner                
1           1        1_1970     CAT3     DEP1       B1              186   
            3        1_1970     CAT2     DEP3       B3               50   
            5        1_1970     CAT2     DEP2       B5               57   
                                         DEP3       B5               69   
                                CAT3     DEP4       B5              160   
2           2        1_1970     CAT1     DEP4       B2              155   
6           2        1_1970     CAT2     DEP3       B2               20   
7           1        1_1970     CAT3     DEP1       B1              100   
9           1        1_1970     CAT1     DEP4       B1              146   
            2        1_1970     CAT2     DEP2       B2              170   
            3        1_1970     CAT4     DEP3       B3              105   
10          1        1_1970     CAT1     DEP2       B1              239   
            2        1_1970     CAT1     DEP4       B2              183   
                                CAT3     DEP1       B2              163   
            4        1_1970     CAT2     DEP3       B4              147   

                                                            Total_units  \
customer_id store_id month_year category department banner                
1           1        1_1970     CAT3     DEP1       B1                4   
            3        1_1970     CAT2     DEP3       B3                4   
            5        1_1970     CAT2     DEP2       B5                2   
                                         DEP3       B5                3   
                                CAT3     DEP4       B5                3   
2           2        1_1970     CAT1     DEP4       B2                1   
6           2        1_1970     CAT2     DEP3       B2                3   
7           1        1_1970     CAT3     DEP1       B1                3   
9           1        1_1970     CAT1     DEP4       B1                4   
            2        1_1970     CAT2     DEP2       B2                1   
            3        1_1970     CAT4     DEP3       B3                3   
10          1        1_1970     CAT1     DEP2       B1                2   
            2        1_1970     CAT1     DEP4       B2                4   
                                CAT3     DEP1       B2                8   
            4        1_1970     CAT2     DEP3       B4                2   

                                                            Visits_count  \
customer_id store_id month_year category department banner                 
1           1        1_1970     CAT3     DEP1       B1                 1   
            3        1_1970     CAT2     DEP3       B3                 1   
            5        1_1970     CAT2     DEP2       B5                 1   
                                         DEP3       B5                 1   
                                CAT3     DEP4       B5                 1   
2           2        1_1970     CAT1     DEP4       B2                 1   
6           2        1_1970     CAT2     DEP3       B2                 1   
7           1        1_1970     CAT3     DEP1       B1                 1   
9           1        1_1970     CAT1     DEP4       B1                 1   
            2        1_1970     CAT2     DEP2       B2                 1   
            3        1_1970     CAT4     DEP3       B3                 1   
10          1        1_1970     CAT1     DEP2       B1                 2   
            2        1_1970     CAT1     DEP4       B2                 2   
                                CAT3     DEP1       B2                 2   
            4        1_1970     CAT2     DEP3       B4                 2   

                                                            Products_count  \
customer_id store_id month_year category department banner                   
1           1        1_1970     CAT3     DEP1       B1     

6.	Considering the four tables provided and any additional field/table derived from the last analysis (you will have available two years of historical data), how will you measure customer loyalty segmentation in terms of shopping? How will you approach your idea and what methodology will you choose to segment the customers and validate your results?
Only explain yourself, no code required.


To measure customer loyalty, several procedures are available. In my opinion, some fields give special information that must be taken as priority. In the last table, *customer_metrics*, some fields like 'Visits_count_customer' or just 'Visits_count' allow us to meet the customers with more visits to the instalations, within the given time, depending on whether we need to analyze the visits to a certain store, filtering by department, category or product or not. Sorting these fields would help us to find the customers with more visits to each store. 

We could also take the Last_visit field from the *customer_metrics* table. This field could help us to find if a given customer is still active or if he/she has deserted.

New metrics could be created, as an example, a ratio between the 'Visits_count' and a determined period of time (weeks, months, years) could be found and sorted, to find the most rated-frequent customers, or well, a ratio between the the 'Products_count' and the 'Visits_count'. These two new metrics could tell us how often a customer visits us and how many products he buys per visit, in average. 

Other fields could also be analyzed, such as 'Total_sales' or 'Products_count_customer', which can give us information about what the customer buys, but taking into account that a high value in this field does not necessarily mean that the customer is loyal to our company: The customer could made a big purchase in a single visit.

**To approach my idea**, i would mainly use plots and unsupervised learning, when needed. In this way, I could first examine the dispersion of the data and define possible patterns among it. If needed, unsupervised machine-learning methods like clustering could help me segment different customer groups according to their characteristics. After having trained a classification model, results could be validated, in order to measure the model behavior.